# Matrix reshape lemmas

This notebook contains all matrix reshaping lemmas found during the BLMM project.

## Setup

In [0]:
import numpy as np
import time
import scipy.sparse

### Helper functions

This section contains helper functions used later in the notebook.

In [0]:
def mat2vec(matrix):
  
  #Return vectorised matrix
  return(matrix.transpose().reshape(matrix.shape[0]*matrix.shape[1],1))

def vec2mat(vec): # Only for square matrices
  
  # Return matrix
  return(vec.reshape(np.int64(np.sqrt(vec.shape[0])),np.int64(np.sqrt(vec.shape[0]))).transpose())

# Block vec with row-wise chunks of size n
def matb2vecb(matb,n):
  
  m = matb.shape[0]
  k = matb.shape[1]

  vecb = matb.reshape(m//n, n, k).transpose((1, 0, 2)).reshape(n, m*k//n).transpose().reshape(m//n,n*k)

  #Return vectorised matrix
  return(vecb)

In [0]:
def comMat(a, b):

  # Example matrix with unique elements
  m = np.arange(a*b).reshape(a,b)

  # Vec(m) and Vec(m')
  vecm = mat2vec(m)
  vecmt = mat2vec(m.transpose())

  # Work out mapping between them.
  k=(vecm.transpose()==vecmt).astype(int).transpose()

  return(k)

## Lemmas

### **Lemma 1: Sum of $A_iB_i'$**


Let $A$ and $B$ be matrices of size $(m,k)$, partitioned row-wise into chunks $A_1,...A_{m/n}$ and $B_1,...B_{l}$ of size $(n,k)$ where $m|n$ and $l=m/n$. I.e.

$$A = \begin{bmatrix} A_1 \\ A_2 \\ \vdots \\ A_{l} \end{bmatrix}, B = \begin{bmatrix} B_1 \\ B_2 \\ \vdots \\ B_{l} \end{bmatrix}$$

Then:

$$\sum_{i=1}^l A_iB_i' = ((A')^{r(kl,n)})'(B')^{r(kl,n)}$$

In [0]:
m = 8
n=2
l=m//n
k=4

print('Example Shapes:')
print('m: ', m)
print('k: ', k)
print('l: ', l)
print('n: ', n)

# A and B are random matrices
A = np.random.randn(m,k)
B = np.random.randn(m,k)

t1 = time.time()

# First we evaluate the LHS
sumAB = np.zeros((n,n))
for i in np.arange(m//n):

  # Work out the row-wise chunks
  Ai = A[n*i:n*(i+1),:]
  Bi = B[n*i:n*(i+1),:]

  # Perform the summation
  sumAB = sumAB + Ai @ Bi.transpose()

t2 = time.time()

# We print the time
time1=t2-t1
print('Time (LHS):')
print(time1)

# And the result
print('Result (LHS):')
print(sumAB)

# Now the RHS
t1 = time.time()
sumAB2 = A.transpose().reshape((m*k//n,n)).transpose() @ B.transpose().reshape((m*k//n,n))
t2 = time.time()

# We print the time
time2=t2-t1
print('Time (RHS):')
print(time2)

# And the result
print('Result (RHS):')
print(sumAB2)

# And the difference
print('Difference:')
print(sumAB-sumAB2)


# And the ratio between times
print('time ratio:')
print(time2/time1)

Example Shapes:
m:  8
k:  4
l:  4
n:  2
Time (LHS):
0.007241249084472656
Result (LHS):
[[ 1.46256925 -0.10209768]
 [ 1.88511476  1.96116093]]
Time (RHS):
0.0001621246337890625
Result (RHS):
[[ 1.46256925 -0.10209768]
 [ 1.88511476  1.96116093]]
Difference:
[[-2.22044605e-16 -1.38777878e-16]
 [-4.44089210e-16  2.22044605e-16]]
time ratio:
0.022389042539180823


In [0]:
print(A)

print(A.transpose().reshape((m*k//n,n)).transpose())

[[ 1.08371092 -1.45263535 -0.09399076  0.84927422]
 [-0.41021823  1.36385848  0.23557171  0.09087627]
 [-1.09898401  2.40684285 -0.30538174 -0.32349291]
 [ 1.19170937  0.48921162  0.53209444 -0.17686872]
 [-0.16694237 -0.04981774  0.28025551  1.04162343]
 [ 0.31141766  0.1438598  -0.06216575 -1.24705787]
 [ 1.00542404 -1.352751    0.1985569  -1.46756808]
 [-0.63278181 -1.67700932  0.23345516  0.54106947]]
[[ 1.08371092 -1.09898401 -0.16694237  1.00542404 -1.45263535  2.40684285
  -0.04981774 -1.352751   -0.09399076 -0.30538174  0.28025551  0.1985569
   0.84927422 -0.32349291  1.04162343 -1.46756808]
 [-0.41021823  1.19170937  0.31141766 -0.63278181  1.36385848  0.48921162
   0.1438598  -1.67700932  0.23557171  0.53209444 -0.06216575  0.23345516
   0.09087627 -0.17686872 -1.24705787  0.54106947]]


### **Lemma 2: Sum of $A_iCB_i'$**


Let $A$ and $B$ be matrices of sizes $(m,k_1)$ and $(m,k_2)$ respectively, partitioned row-wise into chunks $A_1,...A_{m/n}$ and $B_1,...B_{l}$ of sizes $(n,k_1)$ and $(m,k_2)$ respectively where $m|n$ and $l=m/n$. I.e.

$$A = \begin{bmatrix} A_1 \\ A_2 \\ \vdots \\ A_{l} \end{bmatrix}, B = \begin{bmatrix} B_1 \\ B_2 \\ \vdots \\ B_{l} \end{bmatrix}$$

Let $C$ be a matrix of size $(k_1, k_2)$. Then:

$$\sum_{i=1}^l A_iCB_i' = ((A')^{r(k_1l,n)})'(CB')^{r(k_1l,n)} = ((C'A')^{r(k_2l,n)})'(B')^{r(k_2l,n)}$$

In [0]:
m = 1000
n=2
l=m//n
k1=100
k2=3

print('Example Shapes:')
print('m: ', m)
print('k1, k2: ', k1, k2)
print('l: ', l)
print('n: ', n)

# A, B and C are random matrices
A = np.random.randn(m,k1)
B = np.random.randn(m,k2)
C = np.random.randn(k1,k2)

t1 = time.time()

# First we evaluate the LHS
sumACB = np.zeros((n,n))
for i in np.arange(m//n):

  # Work out the row-wise chunks
  Ai = A[n*i:n*(i+1),:]
  Bi = B[n*i:n*(i+1),:]

  # Perform the summation
  sumACB = sumACB + Ai @ C @ Bi.transpose()

t2 = time.time()

# We print the time
time1=t2-t1
print('Time (LHS):')
print(time1)

# And the result
print('Result (LHS):')
print(sumACB)

# Now the middle
t1 = time.time()
sumACB2 = A.transpose().reshape((m*k1//n,n)).transpose() @ (C @ B.transpose()).reshape((m*k1//n,n))
t2 = time.time()

# We print the time
time2=t2-t1
print('Time (middle):')
print(time2)

# And the result
print('Result (middle):')
print(sumACB2)

# Now the RHS
t1 = time.time()
sumACB3 = (C.transpose() @ A.transpose()).reshape((m*k2//n,n)).transpose() @ B.transpose().reshape((m*k2//n,n))
t2 = time.time()

# We print the time
time3=t2-t1
print('Time (RHS):')
print(time3)

# And the result
print('Result (RHS):')
print(sumACB3)

# And the difference
print('Differences:')
print(sumACB-sumACB2)
print(sumACB2-sumACB3)
print(sumACB3-sumACB)


# And the ratio between times
print('time ratios:')
print(time2/time1)
print(time3/time2)
print(time1/time3)

Example Shapes:
m:  1000
k1, k2:  100 3
l:  500
n:  2
Time (LHS):
0.004273891448974609
Result (LHS):
[[-137.78129531  158.27541276]
 [ 981.59795074  285.40823091]]
Time (middle):
0.0031974315643310547
Result (middle):
[[-137.78129531  158.27541276]
 [ 981.59795074  285.40823091]]
Time (RHS):
0.0006031990051269531
Result (RHS):
[[-137.78129531  158.27541276]
 [ 981.59795074  285.40823091]]
Differences:
[[ 4.26325641e-13  2.84217094e-14]
 [-3.41060513e-13  2.27373675e-13]]
[[1.42108547e-13 3.12638804e-13]
 [1.25055521e-12 0.00000000e+00]]
[[-5.68434189e-13 -3.41060513e-13]
 [-9.09494702e-13 -2.27373675e-13]]
time ratios:
0.7481312060693964
0.18865110729997764
7.085375494071147


### **Lemma 3: Sum of $(A_i-B_iC)(A_i-B_iC)'$**


Let $A$ and $B$ be matrices of sizes $(m,k_1)$ and $(m,k_2)$ respectively, partitioned row-wise into chunks $A_1,...A_{m/n}$ and $B_1,...B_{l}$ of sizes $(n,k_1)$ and $(m,k_2)$ respectively where $m|n$ and $l=m/n$. I.e.

$$A = \begin{bmatrix} A_1 \\ A_2 \\ \vdots \\ A_{l} \end{bmatrix}, B = \begin{bmatrix} B_1 \\ B_2 \\ \vdots \\ B_{l} \end{bmatrix}$$

Let $C$ be a matrix of size $(k_1, k_2)$. Then:

$$\sum_{i=1}^l (A_i-B_iC)(A_i-B_iC)'=((A')^{r(k_1l,n)})'(A')^{r(k_1l,n)}$$

### **Lemma 4: Sum of $A_i \otimes B_i$**

Let $A$ and $B$ be matrices of sizes $(m,k_1)$ and $(m,k_2)$ respectively, partitioned row-wise into chunks $A_1,...A_{m/n}$ and $B_1,...B_{l}$ of sizes $(n,k_1)$ and $(n,k_2)$ respectively where $m|n$ and $l=m/n$. I.e.

$$A = \begin{bmatrix} A_1 \\ A_2 \\ \vdots \\ A_{l} \end{bmatrix}, B = \begin{bmatrix} B_1 \\ B_2 \\ \vdots \\ B_{l} \end{bmatrix}$$

Let $a_i=vec(A_i)$ and $b_i=vec(B_i)$ for all $i \in \{1,...,l\}$ and define:

$$\tilde{a} = \begin{bmatrix} a_1 \\ a_2 \\ \vdots \\ a_{l} \end{bmatrix}, \tilde{b} = \begin{bmatrix} b_1 \\ b_2 \\ \vdots \\ b_{l} \end{bmatrix}$$

Then we have the following relationship:

$$\sum_{i=1}^{l} A_i \otimes B_i = ((I_{k_1} \otimes P_{n,k_2} \otimes I_{n})vec(\tilde{b}'\tilde{a}))^{r(n^2,k_1k_2)}$$

In [0]:
m = 10
n=2
l=m//n
k1=6
k2=3

print('Example Shapes:')
print('m: ', m)
print('k1, k2: ', k1, k2)
print('l: ', l)
print('n: ', n)

# A, B and C are random matrices
A = np.random.randn(m,k1)
B = np.random.randn(m,k2)

t1 = time.time()

# First we evaluate the LHS
sumAkB = np.zeros((n**2,k1*k2))
for i in np.arange(m//n):

  # Work out the row-wise chunks
  Ai = A[n*i:n*(i+1),:]
  Bi = B[n*i:n*(i+1),:]

  # Perform the summation
  sumAkB = sumAkB + np.kron(Ai, Bi)

t2 = time.time()
print('Time (LHS):')
print(t2-t1)
print('Shape (LHS):')
print(sumAkB.shape)

# Now for RHS

t1 = time.time()
# This matrix only needs be calculated once
K = comMat(n, k2)
IPI = np.kron(np.kron(np.eye(k1), K), np.eye(n))

t2 = time.time()
atilde = matb2vecb(A,n)
btilde = matb2vecb(B,n)

vecba = mat2vec(btilde.transpose() @ atilde)

RHS_noreshape = IPI @ vecba 

RHS_final = RHS_noreshape.reshape(k1*k2,n**2).transpose()


t3 = time.time()
print(t3-t1)
print(t3-t2)

print(np.allclose(RHS_final, sumAkB))
print(RHS_final)
print(sumAkB)

print(K)

print(IPI @ np.arange(IPI.shape[1]))

perm = (IPI @ np.arange(IPI.shape[1])).astype(int)
print(perm)

t1 = time.time()
vecba[perm[:]]
t2 = time.time()
print(t2-t1)
t1 = time.time()
IPI @ vecba
t2 = time.time()
print(t2-t1)
#print(vecba[perm[:]]-IPI @ vecba)

print(K @ np.arange(K.shape[0]))

Example Shapes:
m:  10
k1, k2:  6 3
l:  5
n:  2
Time (LHS):
0.0006401538848876953
Shape (LHS):
(4, 18)
0.0006539821624755859
0.00022649765014648438
True
[[-0.46929785 -0.14005249  0.46835193  1.03272659 -0.84348491  1.41662908
  -2.42130386  1.63738497 -0.93094915 -2.57781476  1.22905981  1.23835086
   0.55721421  0.96485453 -0.33331466  1.74734725  1.0179927   0.59351314]
 [-0.93538867  0.54965384 -2.55135927 -0.89320179  0.94484317 -0.67963582
   1.83020913 -0.95450988 -0.69598713  3.63306286  1.0887329  -2.05970943
  -0.62434324 -0.8868745   0.88579347  0.3527816  -0.28820265  2.56520035]
 [-0.31696956  1.2671171  -1.47497934 -0.2629326   0.06655848 -0.61607683
   2.67372899 -0.17148013 -0.55793188 -0.16629421 -0.86069956  3.0082058
   0.12854007 -0.77185424  2.28103104 -1.32152269 -0.4459417  -1.40321593]
 [ 0.83171211 -1.04839483  1.4851392   2.3701172  -0.21876412  2.70225157
  -4.80767887 -2.0590688   1.69536008  3.66897525  1.63042294  1.6820762
  -0.39964563  0.7597807  -1.154

### **Lemma 5: Double sum of $A_iB_j' \otimes C_iD_j'$**


Let $A$ and $C$ be matrices of sizes $(m_1,k_1)$ and $(m_1,k_2)$ respectively, partitioned row-wise into chunks $A_1,...A_{l_1}$ and $C_1,...C_{l_1}$ of sizes $(n_1,k_1)$ and $(n_1,k_2)$ respectively where $m_1|n_1$ and $l_1=m_1/n_1$. I.e.

$$A = \begin{bmatrix} A_1 \\ A_2 \\ \vdots \\ A_{l_1} \end{bmatrix}, C = \begin{bmatrix} C_1 \\ C_2 \\ \vdots \\ C_{l_1} \end{bmatrix}$$

Let $B$ and $D$ be matrices of sizes $(m_2,k_1)$ and $(m_2,k_2)$ respectively, partitioned row-wise into chunks $B_1,...B_{l_2}$ and $D_1,...D_{l_2}$ of sizes $(n_2,k_1)$ and $(n_2,k_2)$ respectively where $m_2|n_2$ and $l_2=m_2/n_2$. I.e.

$$B = \begin{bmatrix} B_1 \\ \vdots \\ B_{l_2} \end{bmatrix}, D = \begin{bmatrix} D_1\\ \vdots \\ D_{l_2} \end{bmatrix}$$

Then:

$$\sum_{j=1}^{l_2} \sum_{i=1}^{l_1}A_iB_j' \otimes C_iD_j' = ((I_{k_1} \otimes P_{n_1,k_2} \otimes I_{n_1})vec(\tilde{c}'\tilde{a}))^{r(n_1^2,k_1k_2)}\big[((I_{k_1} \otimes P_{n_2,k_2} \otimes I_{n_2})vec(\tilde{d}'\tilde{b}))^{r(n_2^2,k_1k_2)}\big]'$$

(This follows directly from lemma 4 and the mixed product property of the kronecker product).

In [0]:
# Dimensions
import scipy.sparse

# Partition sizes
n1 = 2
n2 = 2

# Number of rows
m1 = 1000
m2 = 1200

# Number of columns
k1 = 1000
k2 = 1000

# Matrices 

A = np.random.randn(m1,k1)
B = np.random.randn(m2,k1)
C = np.random.randn(m1,k2)
D = np.random.randn(m2,k2)

runningSum = np.zeros((n1**2,n2**2))
t1 = time.time()
# Work out LHS
for j in np.arange(m2//n2):

  Bj = B[n2*j:n2*(j+1),:]
  Dj = D[n2*j:n2*(j+1),:]

  for i in np.arange(m1//n1):

    Ai = A[n1*i:n1*(i+1),:]
    Ci = C[n1*i:n1*(i+1),:]

    runningSum = runningSum + np.kron(Ai @ Bj.transpose(), Ci @ Dj.transpose())

t2 = time.time()
print('Time (LHS)')
print(t2-t1)

# Now for RHS

t1 = time.time()
# This matrix only needs be calculated once
Kmat1 = scipy.sparse.csr_matrix(comMat(n1, k2))
IPI1 = scipy.sparse.kron(scipy.sparse.kron(scipy.sparse.identity(k1), Kmat1), scipy.sparse.identity(n1))
Kmat2 = scipy.sparse.csr_matrix(comMat(n2, k2))
IPI2 = scipy.sparse.kron(scipy.sparse.kron(scipy.sparse.identity(k1), Kmat2), scipy.sparse.identity(n2))

t2 = time.time()
atilde = matb2vecb(A,n1)
btilde = matb2vecb(B,n2)
ctilde = matb2vecb(C,n1)
dtilde = matb2vecb(D,n2)

vecca = mat2vec(ctilde.transpose() @ atilde)
vecdb = mat2vec(dtilde.transpose() @ btilde)

RHS1_noreshape = IPI1 @ vecca 
RHS2_noreshape = IPI2 @ vecdb 

RHS1_final = RHS1_noreshape.reshape(k1*k2,n1**2).transpose()
RHS2_final = RHS2_noreshape.reshape(k1*k2,n2**2)

RHS_final = RHS1_final @ RHS2_final

t3 = time.time()
print('Time (preprocessing)')
print(t2-t1)
print('Time (RHS)')
print(t3-t2)

print(np.allclose(RHS_final, runningSum))


Time (LHS)
10.831757307052612
Time (preprocessing)
0.437467098236084
Time (RHS)
0.45050811767578125
True


### **Lemma 6: Double sum of $A_{i,j} \otimes B_iC_j'$**


Let $A$ be a matrix of size $(m_1,m_2)$, partitioned block-wise into chunks $\{A_{ij}\}_{i=1,...l_1}^{j=1...l_2}$ of size $(n_1,n_2)$ where $m_1|n_1$ with $l_1=m_1/n_1$ $m_2|n_2$ with $l_2=m_2/n_2$. I.e.

$$A = \begin{bmatrix} A_{11} & A_{12} & ... & A_{1l_2} \\ A_{21} & A_{22} & ... & A_{2l_2} \\ \vdots \\ A_{l_11} & A_{l_12} & ... & A_{l_1l_2} \end{bmatrix}$$

Let $B$ and $C$ be matrices of sizes $(m_1,k)$ and $(m_2,k)$ respectively, partitioned row-wise into chunks $B_1,...B_{l_1}$ and $C_1,...C_{l_2}$ of sizes $(n_1,k)$ and $(n_2,k)$ respectively. I.e.

$$B = \begin{bmatrix} B_1 \\ \vdots \\ B_{l_1} \end{bmatrix}, C = \begin{bmatrix} C_1 \\ C_2 \\ \vdots \\ C_{l_2} \end{bmatrix}$$

Then:

$$\sum_{j=1}^{l_2} \sum_{i=1}^{l_1}A_{ij} \otimes B_iC_j' = ((I_{n_2} \otimes P_{n_1,n_2} \otimes I_{n_1})vec(\widetilde{(bc')}'\tilde{a}))^{r(n_1^2,n_2^2)}$$

$$\widetilde{(bc')}=vecb(BC')$$
$$\widetilde{a}=vecb(A)$$

In [0]:
n1 = 2
n2 = 2
m1 = 400
m2 = 800
k=100
l1 = m1//n1
l2 = m2//n2

# Create A
A = np.random.randn(m1,m2)

B = np.random.randn(m1,k)
C = np.random.randn(m2,k)

runningSum = np.zeros((n1**2,n2**2))

# LHS calculation
t1 = time.time()
for i in np.arange(l1):
  for j in np.arange(l2):

    Aij = A[n1*i:n1*(i+1),n2*j:n2*(j+1)]
    Bi = B[n1*i:n1*(i+1),:]
    Cj = C[n2*j:n2*(j+1),:]

    runningSum = runningSum + np.kron(Aij, Bi @ Cj.transpose())

t2 = time.time()
print(t2-t1)

# RHS calculation
t1 = time.time()

# Change A to stacked form
As = A.reshape((m1//n1,n1,m2//n2,n2)).transpose(0,2,1,3).reshape(m1*m2//n2,n2)

# B times C transpose
BCt = B @ C.transpose()

# B times C transpose (stacked)
BCts = BCt.reshape((m1//n1,n1,m2//n2,n2)).transpose(0,2,1,3).reshape(m1*m2//n2,n2)

# This matrix only needs be calculated once
K = comMat(n1, n2)
IPI = scipy.sparse.kron(scipy.sparse.kron(scipy.sparse.identity(n2), K), scipy.sparse.identity(n1))

t2 = time.time()
atilde = matb2vecb(As,n1)
bctilde = matb2vecb(BCts,n1)

vecbca = mat2vec(bctilde.transpose() @ atilde)

RHS_noreshape = IPI @ vecbca 

RHS_final = RHS_noreshape.reshape(n1**2,n2**2).transpose()


t3 = time.time()
print(t3-t1)
print(t3-t2)

print(np.allclose(RHS_final, runningSum))
print(RHS_final)
print(runningSum)

2.559058666229248
0.022186994552612305
0.010087013244628906
True
[[  960.83824663 -4552.14415745  2682.444826   -3112.50553303]
 [  727.74973788  4670.61399506  1346.77220663  2369.18564613]
 [-2549.31467565  -377.15571587  5012.98849248 -2177.04529151]
 [ -340.64929828 -4999.05418749  4928.98542478 -1515.14822943]]
[[  960.83824663 -4552.14415745  2682.444826   -3112.50553303]
 [  727.74973788  4670.61399506  1346.77220663  2369.18564613]
 [-2549.31467565  -377.15571587  5012.98849248 -2177.04529151]
 [ -340.64929828 -4999.05418749  4928.98542478 -1515.14822943]]


## Comments

This section contains comments relating the lemmas above to BLMM

### Derivative of $l$ with respect to $D_k$

We already have that; $$\frac{\partial l}{\partial D_k} = \frac{1}{2}\sum_{j=1}^{l_k}(T_{(k,j)}u)(T_{(k,j)}u)' - \frac{1}{2}\sum_{j=1}^kT_{(k,j)}T_{(k,j)}'$$


Consider the first term, given by:
$$\frac{1}{2}\sum_{j=1}^{l_k}(T_{(k,j)}u)(T_{(k,j)}u)'$$
$$= \sigma^{-2}\frac{1}{2}\sum_{j=1}^{l_k}(Z'_{(k,j)}e-Z'_{(k,j)}ZD(I+Z'ZD)^{-1}Z'e)(e'Z_{(k,j)}-e'ZD(I+Z'ZD)^{-1}Z'Z_{(k,j)})$$
$$ = \sigma^{-2}\frac{1}{2}\sum_{j=1}^{l_k} (A_j - B_j)(A_j-B_j)'$$

Upon letting:
 - $A_j = Z'_{(k,j)}e$ with $A = Z'_{(k)}e$ 
 - $B_j = Z'_{(k,j)}ZD(I+Z'ZD)^{-1}Z'e$ with $B = Z'_{(k)}ZD(I+Z'ZD)^{-1}Z'e$ 

Noting, now, by Lemma 1, that this is equal to:

$$= \sigma^{-2}\frac{1}{2}((A')^{r(l_k,q_k)}-(B')^{r(l_k,q_k)})'((A')^{r(l_k,q_k)}-(B')^{r(l_k,q_k)})$$

$$= \sigma^{-2}\frac{1}{2}\bigg((e'Z_{(k)})^{r(l_k,q_k)}-(e'Z(I+DZ'Z)^{-1}DZ'Z_{(k)})^{r(l_k,q_k)}\bigg)'\bigg((e'Z_{(k)})^{r(l_k,q_k)}-(e'Z(I+DZ'Z)^{-1}DZ'Z_{(k)})^{r(l_k,q_k)}\bigg)$$

This is how the first term is computed in a computationally efficient manner. The second term can be computed by first noting:

$$\frac{1}{2}\sum_{j=1}^{l_k}T_{(k,j)}T_{(k,j)}'=\frac{1}{2}\sum_{j=1}^{l_k}\bigg(Z_{(k,j)}'Z_{(k,j)} - Z_{(k,j)}'ZD(I+Z'ZD)^{-1}Z_{(k,j)}'Z\bigg)$$
$$=\frac{1}{2}\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)} - \frac{1}{2}\sum_{j=1}^{l_k}\bigg( Z_{(k,j)}'ZD(I+Z'ZD)^{-1}Z'Z_{(k,j)}\bigg)$$
$$=\frac{1}{2}\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)} - \frac{1}{2}\sum_{j=1}^{l_k}A_jB_j'$$

Upon letting:
 - $A_j = Z'_{(k,j)}Z$ with $A = Z'_{(k)}Z$ 
 - $B_j = Z'_{(k,j)}Z(I+DZ'Z)^{-1}D$ with $B = Z'_{(k)}Z(I+DZ'Z)^{-1}D$ 

Upon applying lemma 1, we see that this equals;

 $$=\frac{1}{2}\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)} - ((A')^{r(l_k\sum_il_iq_i,q_k)})'(B')^{r(l_k\sum_il_iq_i,q_k)}$$
 $$=\frac{1}{2}\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)} - ((Z'Z_{(k,j)})^{r(l_k\sum_il_iq_i,q_k)})'(D(I+Z'ZD)^{-1}Z_{(k,j)}'Z)^{r(l_k\sum_il_iq_i,q_k)}$$

Note that the first term can be computed prior to any iteration as it does not depend on any parameter estimates and therefore can be ignored here.

### Covariance of derivative of $l$ with respect to $D_k$ and $l$ with respect to $\sigma^2$ 


The covariance of the $\frac{\partial l}{\partial \sigma^2}$ and $\frac{\partial l}{\partial D_k}$ is given by:

$$cov\bigg(\frac{\partial l}{\partial \sigma^2},\frac{\partial l}{\partial D_k}\bigg) = \frac{1}{2\sigma^2}\text{vec}'\bigg(\sum_{j=1}^{l_k}T_{(k,j)}T_{(k,j)}'\bigg)\mathcal{D^{+'}}$$




The second term can be computed by first noting:

$$\sum_{j=1}^{l_k}T_{(k,j)}T_{(k,j)}'=\sum_{j=1}^{l_k}\bigg(Z_{(k,j)}'Z_{(k,j)} - Z_{(k,j)}'ZD(I+Z'ZD)^{-1}Z'Z_{(k,j)}\bigg)$$
$$=\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)} - \sum_{j=1}^{l_k}\bigg( Z_{(k,j)}'ZD(I+Z'ZD)^{-1}Z'Z_{(k,j)}\bigg)$$
$$=\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)} - \sum_{j=1}^{l_k}A_jB_j'$$

Upon letting:
 - $A_j = Z'_{(k,j)}Z$ with $A = Z'_{(k)}Z$ 
 - $B_j = Z'_{(k,j)}Z(I+DZ'Z)^{-1}D$ with $B = Z'_{(k)}Z(I+DZ'Z)^{-1}D$ 

Upon applying lemma 1, we see that this equals;

 $$=\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)} - ((A')^{r(l_k\sum_il_iq_i,q_k)})'(B')^{r(l_k\sum_il_iq_i,q_k)}$$
 $$=\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)} - ((Z'Z_{(k)})^{r(l_k\sum_il_iq_i,q_k)})'(D(I+Z'ZD)^{-1}Z'Z_{(k)})^{r(l_k\sum_il_iq_i,q_k)}$$

Note that the first term can be computed prior to any iteration as it does not depend on any parameter estimates and therefore can be ignored here. Therefore, we have:

$$cov\bigg(\frac{\partial l}{\partial \sigma^2},\frac{\partial l}{\partial D_k}\bigg) = \frac{1}{2\sigma^2}\text{vec}'\bigg(\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)}\bigg)\mathcal{D^{+'}} - \frac{1}{2\sigma^2}\text{vec}'\bigg(((Z'Z_{(k)})^{r(l_k\sum_il_iq_i,q_k)})'(D(I+Z'ZD)^{-1}Z'Z_{(k)})^{r(l_k\sum_il_iq_i,q_k)}\bigg)\mathcal{D^{+'}}$$

### Covariance of derivative of $l$ with respect to $D_k$

The covariance of the $\frac{\partial l}{\partial D_{k_1}}$ and $\frac{\partial l}{\partial D_{k_2}}$ is given by:

$$\text{cov}\bigg(\frac{\delta l}{\delta \text{vech}(D_{k_1})},\frac{\delta l}{\delta \text{vech}(D_{k_2})}\bigg)=\frac{1}{2}\mathcal{D}_{k_1}^+\sum_{j=1}^{l_{k_2}}\sum_{i=1}^{l_{k_1}}(R_{(k_1,k_2,i,j)}\otimes R_{(k_1,k_2, i,j)}
)\mathcal{D}_{k_2}^{+'}$$

Where $R_{(k_1,k_2,i,j)}=Z'_{(k_1,i)}Z_{(k_2,j)}-Z_{(k_1,i)}'ZD(I+Z'ZD)^{-1}Z'Z_{(k_2,j)}$. Let:

 - $A_{i,j}=Z'_{(k_1,i)}Z_{(k_2,j)}$ where $A=Z'Z_{(k_1,k_2)}$.
 - $B_{i,j}=Z_{(k_1,i)}'ZD(I+Z'ZD)^{-1}Z'Z_{(k_2,j)}$

It follows that:

 $$\text{cov}\bigg(\frac{\delta l}{\delta \text{vech}(D_{k_1})},\frac{\delta l}{\delta \text{vech}(D_{k_2})}\bigg)=\frac{1}{2}\mathcal{D}_{k_1}^+\sum_{j=1}^{l_{k_2}}\sum_{i=1}^{l_{k_1}}\bigg((A_{i,j} - B_{i,j})\otimes (A_{i,j} - B_{i,j})\bigg)\mathcal{D}_{k_2}^{+'}$$
 $$=\frac{1}{2}\mathcal{D}_{k_1}^+\bigg(\sum_{j=1}^{l_{k_2}}\sum_{i=1}^{l_{k_1}}(A_{i,j}\otimes A_{i,j}) - \sum_{j=1}^{l_{k_2}}\sum_{i=1}^{l_{k_1}}(A_{i,j}\otimes B_{i,j}) - \sum_{j=1}^{l_{k_2}}\sum_{i=1}^{l_{k_1}}(B_{i,j}\otimes A_{i,j}) - \sum_{j=1}^{l_{k_2}}\sum_{i=1}^{l_{k_1}}(B_{i,j}\otimes B_{i,j})\bigg)\mathcal{D}_{k_2}^{+'}$$

Each of these can be calculated in the below mannor via a variant of lemma 6:


$$\sum_{j=1}^{l_2} \sum_{i=1}^{l_1}A_{i,j} \otimes A_{i,j} = ((I_{q_2} \otimes P_{q_1,q_2} \otimes I_{q_1})vec(\tilde{a}'\tilde{a}))^{r(q_1^2,q_2^2)}$$

$$\sum_{j=1}^{l_2} \sum_{i=1}^{l_1}B_{i,j} \otimes A_{i,j} = ((I_{q_2} \otimes P_{q_1,q_2} \otimes I_{q_1})vec(\tilde{b}'\tilde{a}))^{r(q_1^2,q_2^2)}$$

$$\sum_{j=1}^{l_2} \sum_{i=1}^{l_1}A_{i,j} \otimes B_{i,j} = ((I_{q_2} \otimes P_{q_1,q_2} \otimes I_{q_1})vec(\tilde{a}'\tilde{b}))^{r(q_1^2,q_2^2)}$$

$$\sum_{j=1}^{l_2} \sum_{i=1}^{l_1}B_{i,j} \otimes B_{i,j} = ((I_{q_2} \otimes P_{q_1,q_2} \otimes I_{q_1})vec(\tilde{b}'\tilde{b}))^{r(q_1^2,q_2^2)}$$

Where the lower case letter and tilde notation represents the $vecb(\cdot)$ operator. I.e. if $X$ is partitioned into blocks of size $(n_1 \times n_2)$ like so:

$$X = \begin{bmatrix} X_{1,1} & X_{1,2} & ... & X_{1,l_2} \\ X_{2,1} & X_{2,2} & ... & X_{2,l_2} \\ \vdots \\ X_{l_1,1} & X_{l_1,2} & ... & X_{l_1,l_2} \end{bmatrix}$$

Then $\tilde{x}=vecb^{(n_1,n_2)}(X)$ is given by:

$$\tilde{x} = \begin{bmatrix} vec'(X_{1,1}) \\ vec'(X_{1,2}) \\ \vdots \\ vec'(X_{1,l_2}) \\ vec'(X_{2,1}) \\ \vdots \\ vec'(X_{l_1,l_2}) \end{bmatrix}$$

It is also worth noting that the first sum given above does not involve any estimated parameters so does not need to be recomputed for each iteration and can be computed beforehand.

## Final Functions

This section contains functions, and tests of those functions, intended for use in BLMM, based on the previous lemmas.

### Function ```comMat2D```

This function takes as input;

 - $a$: A postive integer.
 - $b$: A postive integer.

And returns:

 - $K$: The commutation matrix (in sparse format) which maps $vec(A)$ to $vec(A')$ for an arbitrary matrix $A$ of dimensions $(a \times b)$, i.e. $K$ is the unique matrix which satisfies, for all $A$;

 $$K vec(A) = vec(A')$$

In [0]:
def comMat2D(a, b):

  # Example matrix with unique elements
  m = np.arange(a*b).reshape(a,b)

  # Vec(m) and Vec(m')
  vecm = mat2vec(m)
  vecmt = mat2vec(m.transpose())

  # Work out mapping between them.
  K=scipy.sparse.csr_matrix((vecm.transpose()==vecmt).astype(int))

  print(type(K))

  return(K)

The below code tests and times ``comMat2D``.

In [0]:
# Random dimensions
a = np.random.randint(1,10)
b = np.random.randint(1,10)

# Get K
t1 = time.time()
K = comMat2D(a,b)
t2 = time.time()
print(t2-t1)

# Random A
A = np.random.randn(a,b)

print(np.allclose(mat2vec(A.transpose()),K @ mat2vec(A)))

<class 'scipy.sparse.csr.csr_matrix'>
0.0016455650329589844
True


### Function ``block2stacked2D``

This function takes as inputs:

 - `A`: A 2D matrix of size $(m_1 \times m_2)$.
 - `pA`: The size of the block partitions of $A$, e.g. if $A_{i,j}$ is of dimension $(n_1 \times n_2)$ the ``pA=[n1, n2]``.

And returns as output:

 - `As`: The matrix $A$ reshaped to have all blocks $A_{i,j}$ on top of one another. I.e. the below mapping has been performed:

$$A = \begin{bmatrix} A_{1,1} & A_{1,2} & ... & A_{1,l_2} \\ A_{2,1} & A_{2,2} & ... & A_{2,l_2} \\ \vdots \\ A_{l_1,1} & A_{l_1,2} & ... & A_{l_1,l_2} \end{bmatrix} \rightarrow A_s = \begin{bmatrix} A_{1,1} \\ A_{1,2} \\ \vdots \\ A_{1,l_2} \\ A_{2,1} \\ \vdots \\ A_{l_1,l_2} \end{bmatrix}$$

In [0]:
def block2stacked2D(A, pA):

  # Work out shape of A
  m1 = A.shape[0]
  m2 = A.shape[1]

  # Work out shape of As
  n1 = pA[0]
  n2 = pA[1]
  
  # Change A to stacked form
  As = A.reshape((m1//n1,n1,m2//n2,n2)).transpose(0,2,1,3).reshape(m1*m2//n2,n2)

  return(As)

Below is a demonstration/test of this function:

In [0]:
# Random block sizes
n1 = np.random.randint(2,6)
n2 = np.random.randint(2,6)
pA = np.array([n1,n2])

# Random number of blocks
l1 = np.random.randint(1,3)
l2 = np.random.randint(1,3)

# Shape of A
m1 = l1*n1
m2 = l2*n2

# Create A
A = np.random.randn(m1,m2)
print(n1,n2,m1,m2)
print(A.shape)
# Print A
print(A)
print(block2stacked2D(A,pA))

2 2 2 4
(2, 4)
[[-0.84359983  0.02359743 -0.51671481  0.73810572]
 [ 1.47444884  0.10687715  0.77949541 -0.31665928]]
[[-0.84359983  0.02359743]
 [ 1.47444884  0.10687715]
 [-0.51671481  0.73810572]
 [ 0.77949541 -0.31665928]]


### Function ``mat2vecb2D``

The below function takes the following inputs:

 - `mat`: An abritary matrix whose dimensions are multiples of `p[0]` and `p[1]` respectively.
 - `p`. The size of the chunks we are partitioning `mat` into.

And gives the following outputs:

 - `vecb`: A matrix composed of each block of `mat`, converted to row vectors, stacked on top of one another. I.e. for an arbitrary matrix $A$ of appropriate dimensions, $vecb(A)$ is the result of the mapping;

 
$$A = \begin{bmatrix} A_{1,1} & A_{1,2} & ... & A_{1,l_2} \\ A_{2,1} & A_{2,2} & ... & A_{2,l_2} \\ \vdots \\ A_{l_1,1} & A_{l_1,2} & ... & A_{l_1,l_2} \end{bmatrix} \rightarrow \tilde{a} = \begin{bmatrix} vec'(A_{1,1}) \\ vec'(A_{1,2}) \\ \vdots \\ vec'(A_{1,l_2}) \\ vec'(A_{2,1}) \\ \vdots \\ vec'(A_{l_1,l_2}) \end{bmatrix}$$

Where $A_{i,j}$ has dimensions $($ `p[0]` $\times$ `p[1]` $)$ for all $i$ and $j$.

In [0]:
# Block vec with row-wise chunks of size n
def mat2vecb2D(mat,p):

  # Change to stacked block format, if necessary
  if p[1]!=mat.shape[1]:
    mat = block2stacked2D(mat,p)

  # Get height of block.
  n = p[0]
  
  # Work out shape of matrix.
  m = mat.shape[0]
  k = mat.shape[1]

  # Convert to stacked vector format
  vecb = mat.reshape(m//n, n, k).transpose((1, 0, 2)).reshape(n, m*k//n).transpose().reshape(m//n,n*k)

  #Return vecb
  return(vecb)

The below code tests and outputs the above function.

In [0]:
# Random block sizes
n1 = np.random.randint(2,4)
n2 = np.random.randint(2,4)

# Blocksize
p = np.array([n1,n2])

# Random number of blocks
l1 = np.random.randint(2,4)
l2 = np.random.randint(2,4)

# Resultant matrix sizes
m1 = l1*n1
m2 = l2*n2

# Random matrices
A = np.random.randn(m1,m2)

print(A)
print(mat2vecb2D(A,p))

[[-1.57740769  1.38227662 -0.57720426 -0.84919883  0.1548305  -0.88733926
   0.27043042 -0.67335116  0.42995405]
 [-0.24377945  1.37536121 -1.92631884 -0.80035341  1.59826768 -0.67487122
  -1.28118976 -1.1020212   1.48634586]
 [-0.25086025 -0.81378966  0.87509298 -0.35159024 -0.30276207  0.22025793
   0.08196965 -0.18488438  1.38398241]
 [ 0.12276833  0.21697103  0.18332077  0.94029174 -0.36673905 -0.21320611
   0.40749003 -1.04558125  0.75996564]
 [ 0.17896961  0.33436014 -1.04736241  0.8176214   1.10268263 -0.28595167
   0.99506683  0.21785647 -2.061598  ]
 [-2.431536   -0.23439462 -0.36009052  1.25263576  1.23069941 -1.69318962
   0.16099168 -1.03901294 -0.14510291]]
[[-1.57740769 -0.24377945 -0.25086025  1.38227662  1.37536121 -0.81378966
  -0.57720426 -1.92631884  0.87509298]
 [-0.84919883 -0.80035341 -0.35159024  0.1548305   1.59826768 -0.30276207
  -0.88733926 -0.67487122  0.22025793]
 [ 0.27043042 -1.28118976  0.08196965 -0.67335116 -1.1020212  -0.18488438
   0.42995405  1.4863

### Function: ``sumAijBijt2D``

This function takes in the following inputs:

 - A: A 2D matrix of dimension $(m_1 \times m_2)$.
 - B: A 2D matrix of dimension $(m_1' \times m_2)$.
 - pA: The size of the block partitions of $A$, e.g. if $A_{i,j}$ is of dimension $(n_1 \times n_2)$ the ``pA=[n1, n2]``.
 - pB: The size of the block partitions of $B$, e.g. if $B_{i,j}$ is of dimension $(n_1' \times n_2)$ the ``pB=[n1', n2]``.

And gives the following output:

 - S: The sum of the partitions of $A$ multiplied by the transpose of the partitions of $B$; i.e. 
 
 $$\sum_{j=1}^{l_1}\sum_{i=1}^{l_2} A_{i,j}B_{i,j}'$$
  
  where $l_1:=\frac{m_1}{n_1}=\frac{m_1'}{n_1'}$ and $l_2:=\frac{m_2}{n_2}$.

In [0]:
def sumAijBijt2D(A, B, pA, pB):
  
  # Work out second (the common) dimension of the reshaped A and B
  nA = pA[0]
  nB = pB[0]

  # Work out the first (the common) dimension of reshaped A and B
  mA = A.shape[0]*A.shape[1]//nA
  mB = B.shape[0]*B.shape[1]//nB

  # Check mA equals mB
  if mA != mB:
    raise Exception('Matrix dimensions incompatible.')

  # Convert both matrices to stacked block format.
  A = block2stacked2D(A,pA)
  B = block2stacked2D(B,pB)

  # Work out the sum
  S = A.transpose().reshape((mA,nA)).transpose() @ B.transpose().reshape((mB,nB))

  # Return result
  return(S)

Below is a test of the function to check it is outputting as expected. It is also timed.

In [0]:
# Random number of blocks
l1 = np.random.randint(1,3000)
l2 = np.random.randint(1,3000)

# Random blocksizes (the second dimension must be the same for both)
n1 = np.random.randint(1,5)
n1prime = np.random.randint(1,5)
n2 = np.random.randint(1,5)

# Save block sizes
pA = np.array([n1,n2])
pB = np.array([n1prime,n2])

m1 = n1*l1
m1prime = n1prime*l1
m2 = n2*l2

k=100
l1 = m1//n1
l2 = m2//n2

# Create A
A = np.random.randn(m1,m2)
B = np.random.randn(m1prime,m2)

# Calculation by summing
t1 = time.time()

# Initiate empty sum
sumAB = np.zeros((n1,n1prime))
for i in np.arange(m1//n1):
  
  for j in np.arange(m2//n2):

    # Work out the row-wise chunks
    Aij = A[n1*i:n1*(i+1),n2*j:n2*(j+1)]
    Bij = B[n1prime*i:n1prime*(i+1),n2*j:n2*(j+1)]

    # Perform the summation
    sumAB = sumAB + Aij @ Bij.transpose()


t2 = time.time()
print('Time (Summing):', t2-t1)

t1 = time.time()
sumAB2 = sumAijBijt2D(A, B, pA, pB)
t2=time.time()
print('Time (Function):', t2-t1)

print(np.allclose(sumAB,sumAB2))

Time (Summing): 5.755505800247192
Time (Function): 0.2979574203491211
True


### Function ``sumAijKronBij2D``

This function takes in the following inputs:

 - `A`: A 2D matrix of dimension $(m_1 \times m_2)$.
 - `B`: A 2D matrix of dimension $(m_1 \times m_2)$.
 - `p`: The size of the block partitions of $A$ and $B$, e.g. if $A_{i,j}$ and$B_{i,j}$ are of dimension $(n_1 \times n_2)$ the ``pA=[n1, n2]``.
 - `K` (optional): The commutation matrix which, for an arbitrary matrix, $X$, of size $(n_1 \times n_2)$, maps $vec(X)$ to $vec(X')$.

And gives the following output:

 - S: The sum of the partitions of $A$ multiplied by the transpose of the partitions of $B$; i.e. 
 
 $$\sum_{j=1}^{l_1}\sum_{i=1}^{l_2} A_{i,j} \otimes B_{i,j}$$
  
  where $l_1:=\frac{m_1}{n_1}$ and $l_2:=\frac{m_2}{n_2}$.

- `K`: The commutation matrix used for calculation (useful for later computation).

In [0]:
def sumAijKronBij2D(A, B, p, K=None):

  # Check dim A and B and pA and pB all same
  n1 = p[0]
  n2 = p[1]

  # This matrix only needs be calculated once
  if K is None:
    K = comMat2D(n1, n2)
  
  # Work out IPI
  IPI = scipy.sparse.kron(scipy.sparse.kron(scipy.sparse.identity(n2), K.transpose()), scipy.sparse.identity(n1))

  # Convert to vecb format
  atilde = mat2vecb2D(A,p)
  btilde = mat2vecb2D(B,p)

  # Multiply and convert to vector
  vecba = mat2vec(btilde.transpose() @ atilde)
  S_noreshape = IPI @ vecba 

  # Reshape to correct shape
  S = S_noreshape.reshape(n2**2,n1**2).transpose()

  return(S,K)

The below code tests and times the above function.

In [0]:
# Random block sizes
n1 = 2#np.random.randint(2,20)
n2 = 3#np.random.randint(2,20)

# Blocksize
p = np.array([n1,n2])

# Random number of blocks
l1 = 2000#np.random.randint(2,20)
l2 = 3000#np.random.randint(2,20)

# Resultant matrix sizes
m1 = l1*n1
m2 = l2*n2

# Random matrices
A = np.random.randn(m1,m2)
B = np.random.randn(m1,m2)

# Sum version
t1 = time.time()

runningSum = np.zeros((n1**2,n2**2))
# LHS calculation
t1 = time.time()
for i in np.arange(l1):
  for j in np.arange(l2):

    Aij = A[n1*i:n1*(i+1),n2*j:n2*(j+1)]
    Bij = B[n1*i:n1*(i+1),n2*j:n2*(j+1)]

    runningSum = runningSum + np.kron(Aij, Bij)

t2 = time.time()
print('Time (summing): ', t2-t1)

# Now with function
t1 = time.time()
S, K = sumAijKronBij2D(A, B, p)
t2 = time.time()

print('Time (function): ', t2-t1)
print(np.allclose(S,runningSum))

t1 = time.time()
S, _ = sumAijKronBij2D(A, B, p, K)
t2 = time.time()

print('Time (function, with K): ', t2-t1)

print(np.allclose(S,runningSum))


Time (summing):  177.29842138290405
<class 'scipy.sparse.csr.csr_matrix'>
Time (function):  1.1384644508361816
True
Time (function, with K):  0.7348198890686035
True


### Miscellaneous


This section contains miscellaneous observations/ideas.

In [0]:
def comMat2D(a, b):

  # Example matrix with unique elements
  m = np.arange(a*b).reshape(a,b)

  # Vec(m) and Vec(m')
  vecmt = mat2vec(m)
  vecmt = vecmt.reshape(vecmt.shape[0])

  # Get row inds
  rowInds = np.arange(a*b)

  # Get column inds
  colInds = np.argsort(vecmt)
  colInds = colInds.reshape(colInds.shape[0])

  # Work out mapping between them.
  K=scipy.sparse.csr_matrix((np.ones(a*b),(rowInds,colInds)))

  return(K)

In [0]:
# Random dimensions
a = 100#np.random.randint(10,20)
b = 90#np.random.randint(10,20)

# Get K
t1 = time.time()
K = comMat2D(a,b)
t2 = time.time()
print(t2-t1)

# Random A
A = np.random.randn(a,b)

print(K.shape)
print(mat2vec(A.transpose()).shape)
print(mat2vec(A).shape)

print(np.allclose(mat2vec(A.transpose()),K @ mat2vec(A)))

0.002140045166015625
(9000, 9000)
(9000, 1)
(9000, 1)
True


In [0]:
def commutation_matrix_sp(m,n):
  
    row  = np.arange(m*n)
    col  = row.reshape((m, n), order='F').ravel()
    print(col)
    data = np.ones(m*n, dtype=np.int8)
    K = scipy.sparse.csr_matrix((data, (row, col)), shape=(m*n, m*n))
    print(K @ np.arange(K.shape[1]))
    return K

# Random dimensions
a = 100#np.random.randint(10,300)
b = 90#np.random.randint(10,300)

# Get K
t1 = time.time()
K = commutation_matrix_sp(a,b)
t2 = time.time()
print(t2-t1)

# Random A
A = np.random.randn(a,b)

print(K.shape)
print(mat2vec(A.transpose()).shape)
print(mat2vec(A).shape)

print(np.allclose(mat2vec(A.transpose()),K @ mat2vec(A)))

[   0  100  200 ... 8799 8899 8999]
[   0  100  200 ... 8799 8899 8999]
0.003014087677001953
(9000, 9000)
(9000, 1)
(9000, 1)
True


The below function takes as inputs:

 - $k_1$: A positive integer.
 - $k_2$: A positive integer.
 - $n$: A positive integer.

And returns the permutation vector $p$ such that for any matrix $A$

$(I_{k_1} \otimes P_{n,k_2} \otimes I_{n})A=A_p$

Where $A_p$ is the matrix $A$ with $p$ applied to it's rows.

In [0]:
def permOfIPI(k1,k2,n):

  # First we need the permutation represented by matrix P in vector format
  permP = np.arange(n*k2).reshape((k2, n), order='F').ravel()

  # Now we work out the permutation obtained by the first kronecker product (i.e. I kron P)
  permKron1 = np.repeat(np.arange(k1),n*k2)*n*k2+np.tile(permP,k1)

  # Now we work out the final permutation by appplying the second kronecker product (i.e. I kron P kron I)
  p = np.repeat(permKron1,n)*n+np.tile(np.arange(n),n*k1*k2)

  # Return the permutation
  return(p)

In [0]:
m = 10#np.random.randint(10,200)
n=8#np.random.randint(10,200)
k1=5#np.random.randint(10,200)
k2=6#np.random.randint(10,200)

print('Example Shapes:')
print('m: ', m)
print('k1, k2: ', k1, k2)
print('n: ', n)

testMat = np.random.randn(k1*k2*n**2,m)

t1 = time.time()
K = comMat2D(n, k2).toarray()
print(K.shape)
IPI = np.kron(np.kron(np.eye(k1), K), np.eye(n))
result1 = IPI @ testMat
t2 = time.time()
print(t2-t1)
print(IPI.shape[1])

t1 = time.time()
perm = permOfIPI(k1,k2,n)
result2 = testMat[perm,:]
t2 = time.time()
print(t2-t1)

print(np.allclose(result1,result2))

print(K.shape)


Example Shapes:
m:  10
k1, k2:  5 6
n:  8
(48, 48)
0.04104185104370117
1920
0.0003502368927001953
False
(48, 48)


In [0]:
print(permk.shape)

(6,)


In [0]:
permP = np.arange(n*k2).reshape((n, k2), order='F').ravel()

permP2 = K @ np.arange(K.shape[1])

print(permP)
print(permP2)

kron1 = np.kron(np.eye(k1), K)

permKron1 = np.repeat(np.arange(k1),n*k2)*n*k2+np.tile(permP,k1)

print((kron1 @ np.arange(kron1.shape[0])))
print(permKron1)
print((kron1 @ np.arange(kron1.shape[0]))-permKron1)

[ 0  8 16 24 32 40  1  9 17 25 33 41  2 10 18 26 34 42  3 11 19 27 35 43
  4 12 20 28 36 44  5 13 21 29 37 45  6 14 22 30 38 46  7 15 23 31 39 47]
[ 0.  8. 16. 24. 32. 40.  1.  9. 17. 25. 33. 41.  2. 10. 18. 26. 34. 42.
  3. 11. 19. 27. 35. 43.  4. 12. 20. 28. 36. 44.  5. 13. 21. 29. 37. 45.
  6. 14. 22. 30. 38. 46.  7. 15. 23. 31. 39. 47.]
[  0.   8.  16.  24.  32.  40.   1.   9.  17.  25.  33.  41.   2.  10.
  18.  26.  34.  42.   3.  11.  19.  27.  35.  43.   4.  12.  20.  28.
  36.  44.   5.  13.  21.  29.  37.  45.   6.  14.  22.  30.  38.  46.
   7.  15.  23.  31.  39.  47.  48.  56.  64.  72.  80.  88.  49.  57.
  65.  73.  81.  89.  50.  58.  66.  74.  82.  90.  51.  59.  67.  75.
  83.  91.  52.  60.  68.  76.  84.  92.  53.  61.  69.  77.  85.  93.
  54.  62.  70.  78.  86.  94.  55.  63.  71.  79.  87.  95.  96. 104.
 112. 120. 128. 136.  97. 105. 113. 121. 129. 137.  98. 106. 114. 122.
 130. 138.  99. 107. 115. 123. 131. 139. 100. 108. 116. 124. 132. 140.
 101. 109. 117. 12

In [0]:
permKron1 = np.repeat(np.arange(k1),k1)*k1+np.tile(permP,k1)

ValueError: ignored

In [0]:
print(np.tile([1,2],2))

In [0]:
print(type(K))
